# The Role of Climate in Major Power Outages

**Name(s)**: Chang-Yu Lee & Jianrui Zhang

**Website Link**: (your website link)

## Step 0: Environment Setup

#### Python 3
| Version | Colab Default<sup><small>1</small></sup> | Conda DSC80 |
| :--: | :--: | :--: |
| python 3 | 3.11 | 3.12 |

- `io`, `IPython`, `json` are built-in modules.

#### Module
| Version | Colab Default<sup><small>1</small></sup> | Conda DSC80 | Remark |
| :--: | :--: | :--: | :--: |
| branca | 0.8.1 | NA | draw map |
| folium | 0.19.4 | NA | draw map |
| gdown | 5.2.0 | NA | download file from Google Drive |
| geopandas | 1.0.1 | NA | handle geometry json file |
| numpy | 1.26.4 | 2.1.1 | |
| openpyxl | 3.1.5 | NA | handle excel file |
| pandas | 2.2.2 | 2.2.3 | |
| plotly | 5.24.1 | 5.24.1 | |
| requests | 2.32.3 | 2.32.3 | fetch data from internet |

---

<small>1. The default Colab environment was checked on 2025/02/27. The default python version in Colab currnet is 3.11, and it can't be upgraded to 3.12.</small>

In [1]:
import sys

def is_colab():
  return "google.colab" in sys.modules
def is_dsc80():
  return "dsc80" in sys.executable

if is_colab(): # Colab
  !pip install branca==0.8.1 folium==0.19.4 gdown==5.2.0 \
         geopandas==1.0.1 numpy==2.1.1 openpyxl==3.1.5 \
         pandas==2.2.3 plotly==5.24.1 requests==2.32.3
elif is_dsc80(): # Conda
  !conda install -y -n dsc80 -c conda-forge \
      folium=0.19.4 gdown=5.2.0 geopandas=1.0.1 openpyxl=3.1.5
else: # Unknown
  print("\033[91m[Error] Unknown environment!\033[0m")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: folium
    Found existing installation: folium 0.19.5
    Uninstalling folium-0.19.5:
      Successfully uninstalled folium-0.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

Restart the runtime to apply the changes. You don't need to run the above cell again.
- Google Colab: Click **Runtime** $\to$ **Restart Runtime**.
- Jupyter Notebook: Click **Kernel** $\to$ **Restart Kernel**.

After restart, verify the environment.

In [2]:
import sys

def is_colab():
  return "google.colab" in sys.modules
def is_dsc80():
  return "dsc80" in sys.executable

if is_colab():
  version = !pip freeze | grep -E "^(branca|folium|gdown|geopandas|numpy|openpyxl|pandas|plotly|requests)="
  assert version[0] == "branca==0.8.1"
  assert version[1] == "folium==0.19.4"
  assert version[2] == "gdown==5.2.0"
  assert version[3] == "geopandas==1.0.1"
  assert version[4] == "numpy==2.1.1"
  assert version[5] == "openpyxl==3.1.5"
  assert version[6] == "pandas==2.2.3"
  assert version[7] == "plotly==5.24.1"
  assert version[8] == "requests==2.32.3"
  print("Environment is correct!")
elif is_dsc80():
  version = !conda list -n dsc80 | grep -E "^(branca|folium|gdown|geopandas|numpy|openpyxl|pandas|plotly|requests) "
  assert version[0].split()[1] == "0.8.1" # branca
  assert version[1].split()[1] == "0.19.4" # folium
  assert version[2].split()[1] == "5.2.0" # gdown
  assert version[3].split()[1] == "1.0.1" # geopandas
  assert version[4].split()[1] == "2.1.1" # numpy
  assert version[5].split()[1] == "3.1.5" # openpyxl
  assert version[6].split()[1] == "2.2.3" # pandas
  assert version[7].split()[1] == "5.24.1" # plotly
  assert version[8].split()[1] == "2.32.3" # requests
  print("Environment is correct!")

Environment is correct!


Then, we load all things we need later.

In [66]:
import branca
import folium
import gdown
import geopandas
import numpy as np
from openpyxl import load_workbook
import pandas as pd
import requests

# Built-in modules
import io
from IPython.display import display, HTML
import json

# Plotting modules
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Environment detection
if is_colab():
  pio.renderers.default = "colab"
elif is_dsc80():
  pio.renderers.default = "iframe"

# Plotting setting
pio.templates["dsc80"] = go.layout.Template(
  layout = dict(
    margin = dict(l = 30, r = 30, t = 70, b = 30),
    autosize = True,
    width = 800,
    height = 400,
    xaxis = dict(showgrid = True),
    yaxis = dict(showgrid = True),
    title = dict(x = 0.5, y = 0.97)
  )
)
pio.templates.default = "simple_white+dsc80"

# USA map data (48 states)

# Relationship between "name", "alpha 2 code", "climate region" of lower 48 states.
# Adapted from "https://gist.githubusercontent.com/tvpmb/4734703/raw/b54d03154c339ed3047c66fefcece4727dfc931a/US%2520State%2520List"
!gdown "https://drive.google.com/uc?id=1rYV5klLfePeVr7p056Y9V0x6o1n1LcM0" -O "name_alpha2_climate_lower48only.json"

# Relationship between "state boundaries", "name", of lower 48 states.
# Original source: https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json
!gdown "https://drive.google.com/uc?id=1yqxZgyvyYih2tjFbE3zzP_c9-KSaI5nQ" -O "name_geometry_lower48only.json"

with open("name_alpha2_climate_lower48only.json", "r") as file:
  name_alpha2_climate_json = json.load(file)
with open("name_geometry_lower48only.json", "r") as file:
  geo_name_json = json.load(file)

df_name_alpha2_climate = pd.DataFrame(name_alpha2_climate_json)
df_geo_name = geopandas.GeoDataFrame.from_features(geo_name_json, crs="EPSG:4326")
df_lower48 = df_geo_name.merge(df_name_alpha2_climate, how = "left", left_on = "name", right_on = "name")
df_lower48["geometry"] = df_lower48.geometry.simplify(0.05)
df_lower48.head()

Downloading...
From: https://drive.google.com/uc?id=1rYV5klLfePeVr7p056Y9V0x6o1n1LcM0
To: /content/name_alpha2_climate_lower48only.json
100% 3.23k/3.23k [00:00<00:00, 12.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yqxZgyvyYih2tjFbE3zzP_c9-KSaI5nQ
To: /content/name_geometry_lower48only.json
100% 60.6k/60.6k [00:00<00:00, 88.8MB/s]


,geometry,name,alpha-2,climate.region
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",Alabama,AL,Southeast
1,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",Arizona,AZ,Southwest
2,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas,AR,South
3,"POLYGON ((-123.23326 42.00619, -120.00186 41.9...",California,CA,West
4,"POLYGON ((-107.91973 41.00391, -102.05393 41.0...",Colorado,CO,Southwest


## Step 1: Introduction

In this project, we want to examine the relationship between climate and massive power outages<sup><small>1</small></sup>.

As shown in Figure 1-1, from January 2001 to July 2016, among the recorded 1,534 major power outages, "severe weather" accounted for nearly 50% of the causes compared to other factors. Therefore, if we can successfully predict the occurrence of severe weather events, both in time and space, that lead to major power outages, and take preventive measures in advance, the frequency of such outages can be significantly reduced.

<center><img src="https://i.imgur.com/51u8Cyo.png" height="400px"></center>
<center><strong>Figure 1-1.</strong> Cause of massive power outage. Copied from Figure 2-1.</center>

The dataset was provided by Laboratory for Advancing Sustainable Critical Infrastructure in Purdue University. You can access the original excel file from [here](https://engineering.purdue.edu/LASCI/research-data/outages). Table 1<sup><small>2</small></sup> shows the description of each variable used in this project.

<center><img src="https://i.imgur.com/2ZRNYV5.png" height="400px"></center>
<center><strong>Figure 1-2.</strong> Part of the original excel file.</center>

<center><table>
  <thead>
    <tr>
      <th>Variable Type</th>
      <th>Variable Name</th>
      <th>Description</th>
      <th>Remark</th>
    </tr>
  </thead>
  <tbody>
    <tr><td rowspan="2">General Information</td><td>YEAR</td><td>Indicates the year when the outage event occurred</td><td></td></tr>
    <tr><td>MONTH</td><td>Indicates the month when the outage event occurred</td><td></td></tr>
    <tr><td rowspan="3">Geographic Areas</td><td>U.S._STATE</td><td>Represents all the states in the continental U.S.</td><td></td></tr>
    <tr><td>POSTAL.CODE</td><td>Represents the postal code of the U.S. states</td><td></td></tr>
    <tr><td>NERC.REGION</td><td>The North American Electric Reliability Corporation (NERC) regions involved in the outage event</td><td></td></tr>
    <tr><td rowspan="3">Regional Climate Information</td><td>CLIMATE.REGION</td><td>U.S. Climate regions as specified by the National Centers for Environmental Information</td><td></td></tr>
    <tr><td>ANOMALY.LEVEL</td><td>Represents the oceanic El Niño/La Niña (ONI) index referring to cold and warm episodes</td><td></td></tr>
    <tr><td>CLIMATE.CATEGORY</td><td>Climate episodes classified as “Warm”, “Cold”, or “Normal” based on ONI thresholds</td><td></td></tr>
    <tr><td rowspan="6">Outage Events Information</td><td>OUTAGE.START.DATE</td><td>Day of the year when the outage event started</td><td>Removed</td></tr>
    <tr><td>OUTAGE.START.TIME</td><td>Time of the day when the outage event started</td><td>Removed</td></tr>
    <tr><td>OUTAGE.RESTORATION.DATE</td><td>Day of the year when power was restored</td><td>Removed</td></tr>
    <tr><td>OUTAGE.RESTORATION.TIME</td><td>Time of the day when power was restored</td><td>Removed</td></tr>
    <tr><td>OUTAGE.START</td><td>Time when the outage event started</td><td>Added. Combined from <code>OUTAGE.START.DATE</code> and <code>OUTAGE.START.TIME</code></td></tr>
    <tr><td>OUTAGE.RESTORATION</td><td>Time when power was restored</td><td>Added. Combined from <code>OUTAGE.RESTORATION.DATE</code> and <code>OUTAGE.RESTORATION.TIME</code></td></tr>
    <tr><td rowspan="3">Cause of the Event</td><td>CAUSE.CATEGORY</td><td>Categories of all the events causing major power outages</td><td></td></tr>
    <tr><td>CAUSE.CATEGORY.DETAIL</td><td>Detailed description of event categories causing major power outages</td><td></td></tr>
    <tr><td>HURRICANE.NAMES</td><td>Name of the hurricane if the outage was due to a hurricane</td><td></td></tr>
  </tbody>
</table></center>

<center><strong>Table 1</strong> Variable descriptions.</center>

---

<small>1. As defined by the Department of Energy, the major outages refer to those that impacted atleast 50,000 customers or caused an unplanned firm load loss of atleast 300 MW.</small><br>
<small>2. The table is adapted from this [article](https://www.sciencedirect.com/science/article/pii/S2352340918307182).
</small>



## Step 2-1: Data Cleaning

### (1) Download dataset

In [4]:
!wget -O "outage_origin.xlsx" "https://engineering.purdue.edu/LASCI/research-data/outages/outage.xlsx"

--2025-03-08 01:33:23--  https://engineering.purdue.edu/LASCI/research-data/outages/outage.xlsx
Resolving engineering.purdue.edu (engineering.purdue.edu)... 128.46.104.20
Connecting to engineering.purdue.edu (engineering.purdue.edu)|128.46.104.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 660364 (645K) [application/vnd.ms-excel]
Saving to: ‘outage_origin.xlsx’

outage_origin.xlsx  100%[===================>] 644.89K   749KB/s    in 0.9s    

2025-03-08 01:33:25 (749 KB/s) - ‘outage_origin.xlsx’ saved [660364/660364]



### (2) Remove description

As shown in Figure 1-2, the original excel file contains some description, which are not need for the program. So we remove them here.


In [5]:
wb = load_workbook("./outage_origin.xlsx")
ws = wb.worksheets[0]
for row in sorted([1, 2, 3, 4, 5, 7], reverse=True):
  ws.delete_rows(row)
wb.save("./outage.xlsx")

### (3) Keep relevant variables & Handle data type

Also, since there're some missing values in the excel file, the default `pd.read_excel()` function can't detect the data type correctly. We explicitly specify the data type of each column. Note that data type `Int64`, `Float64` can hold `NaN` value.

For the datetime columns `OUTAGE."{START|RESTORATION}.{DATE|TIME}`, we will handle them later.

In [6]:
# The columns to load
usecols_list = ["OBS", "YEAR", "MONTH", "U.S._STATE", "POSTAL.CODE",
         "NERC.REGION", "CLIMATE.REGION", "ANOMALY.LEVEL",
         "CLIMATE.CATEGORY", "OUTAGE.START.DATE", "OUTAGE.START.TIME",
         "OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME",
         "CAUSE.CATEGORY", "CAUSE.CATEGORY.DETAIL", "HURRICANE.NAMES"]

# Correct types of each column
dtype_dict = {
  "OBS": "Int64",
  "YEAR": "Int64",
  "MONTH": "Int64",
  "U.S._STATE": str,
  "POSTAL.CODE": str,
  "NERC.REGION": str,
  "CLIMATE.REGION": str,
  "ANOMALY.LEVEL": "Float64",
  "CLIMATE.CATEGORY": str,
  "OUTAGE.START.DATE": str,
  "OUTAGE.START.TIME": str,
  "OUTAGE.RESTORATION.DATE": str,
  "OUTAGE.RESTORATION.TIME": str,
  "CAUSE.CATEGORY": str,
  "CAUSE.CATEGORY.DETAIL": str,
  "HURRICANE.NAMES": str
}

df_2_1 = pd.read_excel("outage.xlsx",
             usecols = usecols_list,
             dtype = dtype_dict)

df_2_1.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,NaN,NaN


We handle the datetime columns here. Combine `OUTAGE.{START|RESTORATION}.DATE` and `OUTAGE.{START|RESTORATION}.TIME` into a new Datetime column `OUTAGE.{START|RESTORATION}`, then drop the old those.

In [7]:
df_2_2 = df_2_1.copy()
df_2_2.insert(loc = df_2_2.columns.get_loc("CLIMATE.CATEGORY") + 1,
       column = "OUTAGE.RESTORATION",
       value = pd.to_datetime(df_2_2["OUTAGE.RESTORATION.DATE"].str.split().str[0] + " " + df_2_2["OUTAGE.RESTORATION.TIME"] ,errors="coerce"))
df_2_2.insert(loc = df_2_2.columns.get_loc("CLIMATE.CATEGORY") + 1,
       column = "OUTAGE.START",
       value = pd.to_datetime(df_2_2["OUTAGE.START.DATE"].str.split().str[0] + " " + df_2_2["OUTAGE.START.TIME"] ,errors="coerce"))
df_2_2 = df_2_2.drop(columns=["OUTAGE.START.DATE", "OUTAGE.START.TIME", "OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME"])
df_2_2.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START,OUTAGE.RESTORATION,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,NaN,NaN
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,vandalism,NaN
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,heavy wind,NaN
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,thunderstorm,NaN
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,NaN,NaN


In this study, we exculde "Alaska", "Hawaii", and "District of Columbia"

In [98]:
df_2_3 = df_2_2[~df_2_2["POSTAL.CODE"].isin(["AK", "HI", "DC"])]

### (4) At a glimpse: Importance of severe weather

What percentage of severe weather was the cause of the massive power outage?

In [99]:
category_counts = df_2_3["CAUSE.CATEGORY"].value_counts()
fig_2_1 = go.Figure(data=[go.Pie(labels=category_counts.index, values=category_counts.values)])
fig_2_1.update_layout(title_text="Cause of Massive Power Outage")
display(HTML("<center>" + fig_2_1.to_html(full_html=False) + "</center><br><center><strong>Figure 2-1.</strong> Part of the original excel file.</center>"))

Output hidden; open in https://colab.research.google.com to view.

Figure 1-2 shows that nearly 50% of the massive power outages were due to severe weather. It's definitely worth a in-depth study.

Then, we remove the record caused by other reasons for the rest of the project.

In [100]:
df_2_4 = df_2_3[df_2_3['CAUSE.CATEGORY'] == "severe weather"]
df_2_4 = df_2_4.reset_index(drop = True)
print(df_2_4.shape)
df_2_4.head()

(750, 14)


,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START,OUTAGE.RESTORATION,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,NaN,NaN
1,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,heavy wind,NaN
2,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,thunderstorm,NaN
3,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,NaN,NaN
4,6,2010,11,Minnesota,MN,MRO,East North Central,-1.4,cold,2010-11-13 15:00:00,2010-11-14 22:00:00,severe weather,winter storm,NaN


### (5) Missing values

Before we dig further for the exploration, we need to deal with missing values. Which column has it?

In [12]:
na_cols = df_2_4.columns[df_2_4.isna().any()]
df_2_4[na_cols].isna().mean()

,0
MONTH,0.005333
ANOMALY.LEVEL,0.005333
CLIMATE.CATEGORY,0.005333
OUTAGE.START,0.005333
OUTAGE.RESTORATION,0.025333
CAUSE.CATEGORY.DETAIL,0.245333
HURRICANE.NAMES,0.905333


Well, there's quite a lot! Where and how many are those missing values?

In [13]:
na_row_indices = {col: df_2_4.loc[df_2_4[col].isna(), "OBS"].tolist() for col in na_cols}
df_2_5 = pd.DataFrame({
  "Column": na_row_indices.keys(),
  "Count": [len(indices) for indices in na_row_indices.values()],
  "Indices": na_row_indices.values()
})
df_2_5

,Column,Count,Indices
0,MONTH,4,"[340, 366, 767, 1507]"
1,ANOMALY.LEVEL,4,"[340, 366, 767, 1507]"
2,CLIMATE.CATEGORY,4,"[340, 366, 767, 1507]"
3,OUTAGE.START,4,"[340, 366, 767, 1507]"
4,OUTAGE.RESTORATION,19,"[283, 340, 351, 366, 431, 433, 444, 458, 767, ..."
5,CAUSE.CATEGORY.DETAIL,184,"[1, 5, 19, 20, 27, 28, 32, 36, 41, 53, 74, 78,..."
6,HURRICANE.NAMES,679,"[1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 19, 20, ..."


From above, we have some observations:
1. `MONTH`, `ANOMALY.LEVEL`, `CLIMATE.CATEGORY` and `OUTAGE.START` are missing simultaneously.
2. (1) is a subset of `OUTAGE.RESTORATION`.

Well, this can't be purely a coincidence, right? What's the relation ship among them? We will leave the analysis until **Step 3: Assessment of Missingness**.



## Step 2-2: Exploratory Data Analysis

### (1) Univariate Analysis

1. `YEAR`: Showing long-term variations. A trend of increase followed by a decrease.
2. `MONTH`: Showing seasonal patterns. A peak in summer (6～8) and a secondary peak in winter (12～2).

In [97]:
# Create subplots
fig_2_2 = make_subplots(
  rows = 2, cols = 2,
  subplot_titles = ["(1) Grouped by Year", "(2) Grouped by Month",
            "(3) Grouped by Climate Category",
            "(4) Grouped by Cause Category Detail"]
)

# Modify figure (1, 1)
year_counts = df_2_4["YEAR"].value_counts().sort_index()
fig_2_2.add_trace(
  go.Scatter(
    x = year_counts.index,
    y = year_counts.values,
    text = year_counts.values,
    textposition = "top center",
    mode = "lines + markers + text",
    line = dict(width = 2),
    marker = dict(size = 8)
  ),
  row = 1, col = 1
)
fig_2_2.update_xaxes(
  title_text = "Year",
  tickmode = "linear",
  row = 1, col = 1
)
fig_2_2.update_yaxes(
  title_text = "Number of Outages",
  row = 1, col = 1
)

# Modify figure (1, 2)
month_counts = df_2_4["MONTH"].value_counts().sort_index()
fig_2_2.add_trace(
  go.Bar(
    x = month_counts.index,
    y = month_counts.values,
    width = 0.5,
    text = month_counts.values,
    textposition = "outside"
  ),
  row = 1, col = 2
)
fig_2_2.update_xaxes(
  title_text = "Month",
  tickmode = "linear",
  tickvals = list(range(1, 13)),
  ticktext = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
  row = 1, col = 2
)
fig_2_2.update_yaxes(
  title_text = "Number of Outages",
  row = 1, col = 2
)

# Update overall layout
fig_2_2.update_layout(
    title_text = "Major Power Outages Caused by Severe Weather",
    showlegend = False,
    width = 1600,
    height = 400
)

# Modify figure (2, 1)
climate_cat_counts = df_2_4["CLIMATE.CATEGORY"].value_counts().sort_index()
fig_2_2.add_trace(
  go.Bar(
    x = climate_cat_counts.index,
    y = climate_cat_counts.values,
    width = 0.2,
    text = climate_cat_counts.values,
    textposition = "outside"
  ),
  row = 2, col = 1
)
fig_2_2.update_xaxes(
  title_text = "Climate Category",
  row = 2, col = 1
)
fig_2_2.update_yaxes(
  title_text = "Number of Outages",
  row = 2, col = 1
)

# Modify figure (2, 2)
cause_cat_counts = df_2_4["CAUSE.CATEGORY.DETAIL"].value_counts().sort_index()
fig_2_2.add_trace(
  go.Bar(
    x = cause_cat_counts.index,
    y = cause_cat_counts.values,
    width = 0.5,
    text = cause_cat_counts.values,
    textposition = "outside"
  ),
  row = 2, col = 2
)
fig_2_2.update_xaxes(
  title_text = "Climate Category",
  row = 2, col = 2
)
fig_2_2.update_yaxes(
  title_text = "Number of Outages",
  row = 2, col = 2
)

# Update overall layout
fig_2_2.update_layout(
    title_text = "Major Power Outages Caused by Severe Weather",
    showlegend = False,
    width = 1600,
    height = 800
)

display(HTML("<center>" + fig_2_2.to_html(full_html=False) + "</center><br><center><strong>Figure 2-2.</strong> Major power outages caused by severe weather. \
<br>(Left) Grouped by year, showing long-term variations. <br>(Right) Grouped by month, showing seasonal patterns.</center>"))

Output hidden; open in https://colab.research.google.com to view.

3. `U.S._STATE`: The most severely affected states are "Michigan", "California", and "Texas". Overall, the affected areas are in the west, south through northeast, but barely no in the central part.
4. `NERC.REGION`: Almost the same trend as (3).

In [91]:
df_alpha2_count = df_2_4["POSTAL.CODE"].value_counts().reset_index()

# To let the map drown correctly. We need to fill the missing states.
missing_states = set(df_lower48["alpha-2"].tolist()) - set(df_alpha2_count["POSTAL.CODE"])
df_missing = pd.DataFrame({"POSTAL.CODE": list(missing_states), "count": [0] * len(missing_states)})
df_alpha2_count = pd.concat([df_alpha2_count, df_missing], ignore_index=True)

# Merge geometry information with count
df_merged = pd.merge(df_lower48, df_alpha2_count, left_on="alpha-2", right_on="POSTAL.CODE").drop(columns=["POSTAL.CODE"])

# Create a new folium map
fig_2_3_1 = folium.Map(
  location = [42, -96],
  zoom_start = 4
)

# Define the scale
linear_scale = branca.colormap.linear.OrRd_09.scale(vmin = 1, vmax = df_merged["count"].max())

# Define the information showed when mouse over it
tooltip_info = folium.GeoJsonTooltip(
  fields = ["name", "count"],
  aliases = ["State:", "Number of Events:"],
  localize = True,
  labels = True,
  style = "background-color: yellow;"
)

# Draw the map
folium.GeoJson(
  data = df_merged,
  fill_opacity = 0.8,
  line_opacity = 1.0,
  style_function = lambda feature: {
    "fillColor": "#D3D3D3" if feature["properties"]["count"] == 0 else linear_scale(feature["properties"]["count"]),
    "color": "black",
    "weight": 2
  },
  highlight_function = lambda x: {"weight": 2, "fillOpacity": 1.0}, # Highlight when mouse over it
  tooltip = tooltip_info
).add_to(fig_2_3_1)

In [92]:
df_climate_count = df_2_4["CLIMATE.REGION"].value_counts().reset_index()

# Merge geometry information with count
df_merged = pd.merge(df_lower48, df_climate_count, left_on="climate.region", right_on="CLIMATE.REGION", how="left").drop(columns=["CLIMATE.REGION"])

# Create a new folium map
fig_2_3_2 = folium.Map(
  location = [42, -96],
  zoom_start = 4
)

# Define the scale
linear_scale = branca.colormap.linear.OrRd_09.scale(vmin = 1, vmax = df_merged["count"].max())

# Define the information showed when mouse over it
tooltip_info = folium.GeoJsonTooltip(
  fields = ["name", "climate.region", "count"],
  aliases = ["State:", "Climate Region:", "Number of Events:"],
  localize = True,
  labels = True,
  style = "background-color: yellow;"
)

# Draw the map
folium.GeoJson(
  data = df_merged,
  fill_opacity = 0.8,
  line_opacity = 1.0,
  style_function = lambda feature: {
    "fillColor": "#D3D3D3" if feature["properties"]["count"] == 0 else linear_scale(feature["properties"]["count"]),
    "color": "black",
    "weight": 2
  },
  highlight_function = lambda x: {"weight": 2, "fillOpacity": 1.0}, # Highlight when mouse over it
  tooltip = tooltip_info
).add_to(fig_2_3_2)

In [93]:
fig = branca.element.Figure()

subplot1 = fig.add_subplot(1, 2, 1)
subplot2 = fig.add_subplot(1, 2, 2)

subplot1.add_child(fig_2_3_1)
subplot2.add_child(fig_2_3_2)

display(HTML("<center>" + fig._repr_html_() + "</center><br><center><strong>Figure 2-3.</strong> Major power outages caused by severe weather. \
<br>(Left) Grouped by states. <br>(Right) Grouped by climate regions.</center>"))

In [ ]:
fig = branca.element.Figure()

for index in range()
subplot1 = fig.add_subplot(1, 2, 1)
subplot2 = fig.add_subplot(1, 2, 2)

subplot1.add_child(fig_2_3_1)
subplot2.add_child(fig_2_3_2)

df_winterstorm_count = df_2_4["POSTAL.CODE"].value_counts().reset_index()

# To let the map drown correctly. We need to fill the missing states.
missing_states = set(df_lower48["alpha-2"].tolist()) - set(df_alpha2_count["POSTAL.CODE"])
df_missing = pd.DataFrame({"POSTAL.CODE": list(missing_states), "count": [0] * len(missing_states)})
df_alpha2_count = pd.concat([df_alpha2_count, df_missing], ignore_index=True)

# Merge geometry information with count
df_merged = pd.merge(df_lower48, df_alpha2_count, left_on="alpha-2", right_on="POSTAL.CODE").drop(columns=["POSTAL.CODE"])

# Create a new folium map
fig_2_3_1 = folium.Map(
  location = [42, -96],
  zoom_start = 4
)

# Define the scale
linear_scale = branca.colormap.linear.OrRd_09.scale(vmin = 1, vmax = df_merged["count"].max())

# Define the information showed when mouse over it
tooltip_info = folium.GeoJsonTooltip(
  fields = ["name", "count"],
  aliases = ["State:", "Number of Events:"],
  localize = True,
  labels = True,
  style = "background-color: yellow;"
)

# Draw the map
folium.GeoJson(
  data = df_merged,
  fill_opacity = 0.8,
  line_opacity = 1.0,
  style_function = lambda feature: {
    "fillColor": "#D3D3D3" if feature["properties"]["count"] == 0 else linear_scale(feature["properties"]["count"]),
    "color": "black",
    "weight": 2
  },
  highlight_function = lambda x: {"weight": 2, "fillOpacity": 1.0}, # Highlight when mouse over it
  tooltip = tooltip_info
).add_to(fig_2_3_1)

display(HTML("<center>" + fig._repr_html_() + "</center><br><center><strong>Figure 2-3.</strong> Major power outages caused by severe weather. \
<br>(Left) Grouped by states. <br>(Right) Grouped by climate regions.</center>"))

### (2) Bivariate Analysis

1. `YEAR` and `MONTH`
2. `STATE` and `MONTH`
3. `CLIMATE.REGION` and `MONTH`
4. `CAUSE.CATEGORY.DETAIL` and `MONTH`

In [65]:
region_list = ["Overall", "Northwest", "West", "East North Central",
        "Northeast", "Central", "Southeast", "South"]

# Create subplots
fig_2_5 = make_subplots(
  rows = 4, cols = 2,
  subplot_titles = ["(1) Overall", "(2) Northwest", "(3) West", "(4) East North Central",
            "(5) Northeast", "(6) Central", "(7) Southeast", "(8) South"],
  vertical_spacing = 0.07,
  horizontal_spacing = 0.1
)

# Modify figure (1, 1): Overall
df_overall_count = df_2_4.groupby(["YEAR", "MONTH"]).size().reset_index(name = "count")
df_overall_count = df_overall_count.pivot(index = "MONTH", columns = "YEAR", values = "count").fillna(0)

fig_2_5.add_trace(
  go.Heatmap(
    x = df_overall_count.columns, # Year
    y = df_overall_count.index, # Month
    z = df_overall_count.values, # Count
    colorscale = "gray",
    showscale = False
  ),
  row = 1, col = 1
)

# Modify figure (1, 2) ~ (4, 2): Regional
for index in range(2, 9):
  df_region_count = df_2_4[df_2_4["CLIMATE.REGION"] == region_list[index - 1]]
  df_region_count = df_region_count.groupby(["YEAR", "MONTH"]).size().reset_index(name = "count")
  df_region_count = df_region_count.pivot(index = "MONTH", columns = "YEAR", values = "count").fillna(0)

  fig_2_5.add_trace(
    go.Heatmap(
      x = df_region_count.columns, # Year
      y = df_region_count.index, # Month
      z = df_region_count.values, # Count
      colorscale = "gray",
      showscale = False
    ),
    row = (index+1)//2, col = (index+1)%2+1
  )


# Modify all figures
for index in range(1, 9):
  fig_2_5.update_xaxes(
    title_text = "Year",
    tickmode = "linear",
    tickvals = list(range(2000, 2017)),
    row = (index+1)//2, col = (index+1)%2+1
  )
  fig_2_5.update_yaxes(
    title_text = "Month",
    tickmode = "linear",
    tickvals = list(range(1, 13)),
    row = (index+1)//2, col = (index+1)%2+1
  )

# Update overall layout
fig_2_5.update_layout(
  title_text = "Major Power Outages Caused by Severe Weather",
  title_y = 0.98,
  showlegend = False,
  width = 1200,
  height = 1600
)

display(HTML("<center>" + fig_2_5.to_html(full_html=False) + "</center><br><center><strong>Figure 2-2.</strong> Major power outages caused by severe weather. \
(Left) Grouped by year, showing long-term variations. (Right) Grouped by month, showing seasonal patterns.</center>"))

Output hidden; open in https://colab.research.google.com to view.

### (3) Interesting Aggregates

## Step 3: Assessment of Missingness

In [ ]:
# TODO

## Step 4: Hypothesis Testing

-

In [ ]:
# TODO

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO

## Appendix

### A. Map of U.S Climate Regions

Through climate analysis, National Centers for Environmental Information scientists have identified nine climatically consistent regions within the contiguous United States which are useful for putting current climate anomalies into a historical perspective (Karl and Koss, 1984).

Adapted from [NCEI](https://www.ncei.noaa.gov/access/monitoring/reference-maps/us-climate-regions).

<center><img src="https://i.imgur.com/J9oc61j.png" height="400px"></center>
<center><strong>Figure A.</strong> Map of U.S climate regions.</center>